In [6]:
import pandas as pd

Performance/auditing experiments: 
    
Test the behavior of different Name Servers using
diagnostic tools, such as dnsperf, dnsping, dnstraceroute, dnseval,
reseval.


## 1) Measure the performance of a Name Server when processing multiple queries. 
Did you notice any variability? Any expected/unexpected behavior?





#### https://github.com/DNS-OARC/dnsperf


For some strange reason dnsperf isn't available on aptitude and you have to build it from source.

In [83]:
res = ! dnsperf  -v -n 1000 -s 1.1.1.1 -d dnsperf-queries.txt

In [85]:
records = [l.split(" ")[1:] for l in res if ">" in l]
# records

In [86]:
df = pd.DataFrame(records, columns=["code", "server", "rr", "latency"])
df["latency"]=  pd.to_numeric(df.latency)

In [87]:
df.groupby("rr").latency.mean()

rr
A        0.056626
HTTPS    0.056006
MX       0.056162
NS       0.056179
RRSIG    0.056231
Name: latency, dtype: float64

In [90]:
df.code.value_counts()

NOERROR    4863
T           137
Name: code, dtype: int64

In [92]:
len(df)

5000

Measure the performance of different Name Servers when processing the same
set of queries. Did the response time vary with the Name Server? Does it depend
on the type of query or on the geographic location of the Name Server?



## Queried Nameservers: Cloudflare, Google, Cisco, Quad9

In [2]:
queries = ! cat dnsperf-queries.txt
queries = [q.split(" ")[1] for q in queries if len(q) >= 2]

In [3]:
queries

['A', 'NS', 'MX', 'HTTPS', 'RRSIG']

In [5]:
for q in queries:
    print(f"dnseval -t {q} -c 50 -f name-servers.txt google.com")
    ! dnseval -t {q} -c 50 -f name-servers.txt google.com
    print()

dnseval -t A -c 50 -f name-servers.txt google.com
server             avg(ms)     min(ms)     max(ms)     stddev(ms)  lost(%)  ttl        flags
-----------------------------------------------------------------------------------------------------------
1.1.1.1            42.097      27.729      270.599     39.231      %2       85         QR -- -- RD RA -- --
8.8.8.8            37.794      26.125      56.677      6.292       %0       113        QR -- -- RD RA -- --
208.67.222.222     52.254      44.522      70.872      5.988       %0       300        QR -- -- RD RA -- --
9.9.9.9            41.650      34.372      62.676      5.794       %0       151        QR -- -- RD RA -- --

dnseval -t NS -c 50 -f name-servers.txt google.com
server             avg(ms)     min(ms)     max(ms)     stddev(ms)  lost(%)  ttl        flags
-----------------------------------------------------------------------------------------------------------
1.1.1.1            35.066      27.112      52.984      6.139    

Check the path followed by your queries using different Name Servers; did you
notice any expected/unexpected behavior?

In [40]:
nameservers = ! cat name-servers.txt
nameservers

['1.1.1.1', '8.8.8.8', '208.67.222.222', '9.9.9.9']

In [41]:
p="putyourpasswordinherethenmakesureyoudontcommititorelsebadthingswillhappen"

In [45]:
for ns in nameservers:
    print(f"sudo -S dnstraceroute -s {ns} google.com")
    ! echo {p} | sudo -S dnstraceroute -s {ns} google.com
    print()

sudo -S dnstraceroute -s 1.1.1.1 google.com
[sudo] password for aiman: dnstraceroute DNS: 1.1.1.1:53, hostname: google.com, rdatatype: A
1	_gateway (192.168.1.1) 2.263 ms
2	172.16.129.129 (172.16.129.129) 43.932 ms
3	172.16.129.50 (172.16.129.50) 36.827 ms
4	172.16.14.50 (172.16.14.50) 222.411 ms
5	151.7.82.67 (151.7.82.67) 36.511 ms
6	151.7.80.14 (151.7.80.14) 81.912 ms
7	151.6.0.148 (151.6.0.148) 50.952 ms
8	 *
9	 *
10	188.114.100.11 (188.114.100.11) 1004.079 ms
11	one.one.one.one (1.1.1.1) 54.646 ms

sudo -S dnstraceroute -s 8.8.8.8 google.com
[sudo] password for aiman: dnstraceroute DNS: 8.8.8.8:53, hostname: google.com, rdatatype: A
1	_gateway (192.168.1.1) 2.984 ms
2	172.16.129.129 (172.16.129.129) 44.946 ms
3	172.16.129.50 (172.16.129.50) 37.694 ms
4	172.16.14.50 (172.16.14.50) 33.580 ms
5	151.7.82.67 (151.7.82.67) 47.548 ms
6	151.7.80.14 (151.7.80.14) 39.057 ms
7	151.6.0.148 (151.6.0.148) 55.735 ms
8	151.6.0.159 (151.6.0.159) 197.699 ms
9	74.125.32.80 (74.125.32.80) 44.549 ms
1